In [1]:
spark

StatementMeta(d3c1b951-094c-48e1-919d-e32bc175397b, 18, 7, Finished, Available)

## Loading Data

In [9]:
%%configure -f \
{"conf": {"spark.jars.packages": "com.johnsnowlabs.nlp:spark-nlp_2.12:5.1.2"}}

StatementMeta(d3c1b951-094c-48e1-919d-e32bc175397b, 19, -1, Finished, Available)

Unrecognized options: 

In [41]:
!pip install spark-nlp
!pip install pyspark

StatementMeta(d3c1b951-094c-48e1-919d-e32bc175397b, 19, 37, Finished, Available)

In [53]:
## Necessary Libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.functions import *
import sparknlp
from sparknlp.base import DocumentAssembler, Pipeline
from sparknlp.annotator import (
    SentenceDetector,
    Tokenizer,
    StopWordsCleaner,
    LemmatizerModel
)

StatementMeta(d3c1b951-094c-48e1-919d-e32bc175397b, 19, 49, Finished, Available)

In [12]:
workspace_default_storage_account = "aml6214727930"
workspace_default_container = "azureml-blobstore-6653633b-3460-4381-9199-d9e0f368353c"

workspace_wasbs_base_url = (
    f"wasbs://{workspace_default_container}@{workspace_default_storage_account}.blob.core.windows.net/"
)


StatementMeta(d3c1b951-094c-48e1-919d-e32bc175397b, 19, 8, Finished, Available)

In [49]:
comments_path = "/basketball_comments_clean.parquet"
submissions_path = "/basketball_submissions_clean.parquet"

comments = spark.read.parquet(f"{workspace_wasbs_base_url}{comments_path}")
submissions = spark.read.parquet(f"{workspace_wasbs_base_url}{submissions_path}")

StatementMeta(d3c1b951-094c-48e1-919d-e32bc175397b, 19, 45, Finished, Available)

## NLP Processing

### Data Checks

In [14]:
comments = comments.cache()
submissions = submissions.cache()

StatementMeta(d3c1b951-094c-48e1-919d-e32bc175397b, 19, 10, Finished, Available)

### Cleaning Text Data

In [54]:
document_assembler = (
    DocumentAssembler()
    .setInputCol("body")
    .setOutputCol("document")
)
sentence = (
    SentenceDetectorDLModel.pretrained("sentence_detector_dl", "en")
    .setInputCols(["document"])
    .setOutputCol("sentence")
)
tokenizer = (
    Tokenizer()
    .setInputCols(["sentence"])
    .setOutputCol("token")
)
stop_words = (
    StopWordsCleaner().pretrained("stopwords_iso","en")
    .setInputCols(["token"])
    .setOutputCol("cleanTokens")
)
lemmatizer = (
    LemmatizerModel.pretrained().setInputCols(["cleanTokens"]).setOutputCol("lemma")
)
pipeline = Pipeline(stages=[document_assembler, sentence, tokenizer, stop_words, lemmatizer])

StatementMeta(d3c1b951-094c-48e1-919d-e32bc175397b, 19, 50, Finished, Available)

sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]
stopwords_iso download started this may take some time.
Approximate size to download 2.1 KB
[OK!]
lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]


In [56]:
model = pipeline.fit(comments)
result = model.transform(comments)

StatementMeta(d3c1b951-094c-48e1-919d-e32bc175397b, 19, 52, Finished, Available)

In [59]:
result.select("lemma.result").show(10, truncate = 200)

StatementMeta(d3c1b951-094c-48e1-919d-e32bc175397b, 19, 55, Finished, Available)

+--------------------------------------------------------------------------------------------------------+
|                                                                                                  result|
+--------------------------------------------------------------------------------------------------------+
|                                                                                                  [fall]|
|                                         [lol, forget, football, today, ., ive, work, day, didnt, watch]|
|     [,, half, foul, aren’t, foul, ., like, there’s, multiple, travel, miss, ., team, play, terribly, .]|
|[like, Creighton, Nebraska, land, Iowa, ., Iowa, State, Minnesota, Red/maroon, similar, hard, border, .]|
|                                                                                 [Losing, ,, apparently]|
|                            [Oklahoma, lose, ,, that’s, important, thing, that’s, happen, far, today, .]|
|                                    